# Obteniendo más información de cada caso positivo por COVID-19 en Perú

En este notebook se busca obtener un dataset de todos los **casos positivos** reportados por COVID-19, y agregar información sobre fallecimiento y vacunación recibida (mediante **"UUID"**). Es decir se busca unir las 3 bases de datos, de casos positivos, fallecimientos y vacunación recibida en una sola.

Se considera como **vacunado** a todas las personas que recibieron 2 dosis de vacunación.

## A. Lectura y Procesamiento

### Cargar Librerías

In [2]:
import pandas as pd
import numpy as np
from epiweeks import Week
import dask.dataframe as dd
from datetime import datetime

### Diccionarios para codificar variables

In [3]:
dic_sex = {'MASCULINO': 1,'FEMENINO':2}
dic_met = {'PR': 1, 'PCR':2, 'AG':3}
dic_dep = {
    "AMAZONAS"	:1,
    "ANCASH"	:2,
    "APURIMAC"	:3,   
    "AREQUIPA"	:4,
    "AYACUCHO"	:5,   
    "CAJAMARCA"	:6,
    "CALLAO"	:7,
    "CUSCO"	    :8,
    "HUANCAVELICA":9,	
    "HUANUCO"	:10,	
    "ICA"	    :11,	
    "JUNIN"	    :12,
    "LA LIBERTAD":13,
    "LAMBAYEQUE":14,
    "LIMA"	    :15,
    "LORETO"	:16,
    "MADRE DE DIOS":17,
    "MOQUEGUA"	:18,	
    "PASCO"	    :19,
    "PIURA"	    :20,
    "PUNO"	    :21,
    "SAN MARTIN":22,
    "TACNA"	    :23,
    "TUMBES"	:24,
    "UCAYALI"	:25
}

### Dataset de casos positivos

In [4]:
# Leer dataset
url_cas = '../RawData/positivos_covid.csv'
col_cas = ['FECHA_RESULTADO','EDAD', 'DEPARTAMENTO', 'SEXO', 'id_persona', 'METODODX']
df_cas = pd.read_csv(url_cas, sep = ';', usecols = col_cas, dtype={'EDAD': 'float64', 
                                                                   'FECHA_RESULTADO': 'float64'})

In [5]:
df_cas.head()

,DEPARTAMENTO,METODODX,EDAD,SEXO,FECHA_RESULTADO,id_persona
0,CALLAO,AG,27.0,MASCULINO,20220131.0,24662494.0
1,UCAYALI,AG,39.0,FEMENINO,20220127.0,24662689.0
2,LIMA,PR,25.0,MASCULINO,20201217.0,24662153.0
3,AREQUIPA,PCR,27.0,MASCULINO,20220120.0,24662155.0
4,ICA,PR,20.0,FEMENINO,20200822.0,24662175.0


In [6]:
# TOTAL de casos positivos
len(df_cas)

3534686

In [7]:
# Remover casos con información faltante
df_cas = df_cas.dropna()

# TOTAL de casos positivos sin 'missing values'
len(df_cas)

3465000

In [8]:
# Cambiar 'data type' para reducir tamaño
df_cas = df_cas.astype({'EDAD':'int8', 'FECHA_RESULTADO': 'int32', 'id_persona': 'int32'})

# Correguir datos erróneos encontrados en el anáisis exploradorio de "0_ExploratoryDataAnalysis.ipynb"
df_cas.drop(df_cas[df_cas.SEXO == '.'].index, inplace = True)
df_cas.drop(df_cas[df_cas.EDAD == -79].index, inplace = True)  # un -9 en edad

In [9]:
# TOTAL de casos positivos sin valores erróneos
len(df_cas)

3464999

In [10]:
# Categorizar edad en grupos etáreos
def edad_cat(df, col):
    """Función para categorizar edad en grupos etáreos"""
    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]
    return np.select(conditions, choices, default=np.nan).astype('int8')
df_cas['edad_cat'] = edad_cat(df_cas, 'EDAD') 

# Reemplazar sexo con un code
df_cas['SEXO'].replace(dic_sex, inplace=True)
df_cas['SEXO'] = df_cas['SEXO'].astype('int8')

# Ordenar columnas
df_cas.rename(columns = {'FECHA_RESULTADO':'fecha_resultado',
                         'EDAD': 'edad',
                         'SEXO': 'sexo',
                         'DEPARTAMENTO': 'departamento',
                         'METODODX': 'metododx'}, inplace=True)
df_cas = df_cas[['id_persona','fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento']] \
         .sort_values(by = ['fecha_resultado']) \
         .reset_index(drop = True)

# Codificar variables
df_cas['departamento'] = df_cas['departamento'].map(dic_dep).astype('float')  
df_cas['metododx'].replace(dic_met, inplace=True)
df_cas['metododx'] = df_cas['metododx'].astype('int8')

In [11]:
df_cas.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento
0,34978840,20200306,2,1,25,1,15.0
1,12149681,20200307,2,1,25,1,15.0
2,12211040,20200307,2,1,7,0,15.0
3,29640291,20200307,2,1,29,1,4.0
4,7800643,20200307,2,2,74,6,15.0


### Dataset de fallecidos

In [22]:
# Leer el dataset
url_fal = '../RawData/fallecidos_covid.csv'
col_fal = ['FECHA_FALLECIMIENTO', 'UUID']
df_fal = pd.read_csv(url_fal, sep = ';', usecols= col_fal, dtype = {'FECHA_FALLECIMIENTO':'int32'}, 
                     low_memory=False)

In [23]:
df_fal.head()

,FECHA_FALLECIMIENTO,UUID
0,20210611,24833991.0
1,20210317,24761117.0
2,20210602,24767070.0
3,20210703,24751741.0
4,20210506,24671820.0


In [24]:
# TOTAL de casos positivos
len(df_fal)

211423

In [25]:
# Remover casos con información faltante
df_fal = df_fal.dropna()

# TOTAL de casos positivos sin 'missing values'
len(df_fal)

207538

In [26]:
# Nombrar a todos los casos de fallecimientos con un identificador
df_fal['fallecido'] = 1

# Cambiar de data type para reducir tamaño
df_fal = df_fal.astype({'UUID': 'int32', 'fallecido': 'int8'})

# Transformar a formato fecha para operar los días
df_fal['fecha'] = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d')

# Agregar la semana epidemiológica y el año
df_fal[['year','epi_week']] = df_fal['fecha'].apply(lambda date : Week.fromdate(date) \
                                                                      .weektuple()) \
                                                                      .tolist()

# Ordenar columnas
df_fal.rename(columns = {'FECHA_FALLECIMIENTO':'fecha_fallecimiento', 'UUID':'id_persona'}, inplace=True)

df_fal.head()

,fecha_fallecimiento,id_persona,fallecido,fecha,year,epi_week
0,20210611,24833991,1,2021-06-11,2021,23
1,20210317,24761117,1,2021-03-17,2021,11
2,20210602,24767070,1,2021-06-02,2021,22
3,20210703,24751741,1,2021-07-03,2021,26
4,20210506,24671820,1,2021-05-06,2021,18


### Buscar en el dataset de casos positivos quienes fallecieron y cuándo

In [27]:
# Unir el dataframde de casos positivos con el dataset de fallecidos mediante 'id_persona'
# {‘left’, ‘right’, ‘outer’, ‘inner’, ‘cross’},
df = df_cas.merge(df_fal, on = 'id_persona', how = 'outer')
df['fallecido'].fillna(0, inplace=True)  # Llenar con 0 las personas que no fallecieron en el dataset de casos positivos
df['fallecido'] = df['fallecido'].astype('int8')  # Cambiar 'data type' para reducir tamaño

In [30]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fecha_fallecimiento,fallecido,fecha,year,epi_week
0,34978840,20200306.0,2.0,1.0,25.0,1.0,15.0,NaN,0,NaT,NaN,NaN
1,34978840,20220112.0,2.0,1.0,42.0,3.0,15.0,NaN,0,NaT,NaN,NaN
2,12149681,20200307.0,2.0,1.0,25.0,1.0,15.0,NaN,0,NaT,NaN,NaN
3,12149681,20220111.0,2.0,1.0,73.0,6.0,15.0,NaN,0,NaT,NaN,NaN
4,12211040,20200307.0,2.0,1.0,7.0,0.0,15.0,NaN,0,NaT,NaN,NaN


In [31]:
# Total de casos de casos positivos + información de fallecimiento
len(df)

3570841

In [32]:
# Número de fallecidos en el dataset de casos postivos
len(df[df.fallecido == 1])

214890

**IMPORTANTE**: Notar que algunos casos aparecen más de una vez (reinfecciones de COVID-19)

In [33]:
# Eliminar datasets que ya no serán usados
del df_fal, df_cas

In [34]:
def days_between(d1, d2):
    '''Función que retorna el número de días entre 2 días dados'''
    return abs((d2 - d1).days)

fecha_min = df['fecha'].min()  # Primera fecha reportada en el dataset
fecha_max = df['fecha'].max()  # Última fecha reportada en el dataset

# Obtener el número de días transcurridos a partir de la primera fecha reportada (fecha_min)
df['time_days'] = df['fecha'].apply(lambda row : days_between(row, fecha_min)+1)
df['time_days'] = df['time_days'].fillna(df.time_days.max()).astype('int')

In [35]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fecha_fallecimiento,fallecido,fecha,year,epi_week,time_days
0,34978840,20200306.0,2.0,1.0,25.0,1.0,15.0,NaN,0,NaT,NaN,NaN,738
1,34978840,20220112.0,2.0,1.0,42.0,3.0,15.0,NaN,0,NaT,NaN,NaN,738
2,12149681,20200307.0,2.0,1.0,25.0,1.0,15.0,NaN,0,NaT,NaN,NaN,738
3,12149681,20220111.0,2.0,1.0,73.0,6.0,15.0,NaN,0,NaT,NaN,NaN,738
4,12211040,20200307.0,2.0,1.0,7.0,0.0,15.0,NaN,0,NaT,NaN,NaN,738


In [36]:
# Ordenar columnas
df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento',
         'fallecido', 'time_days', 'fecha_fallecimiento', 'year', 'epi_week']]

# Obtener el año y la semana epidemiológica de cada día obtenido
date_life = Week.fromdate(fecha_max).weektuple()
df['year'] = df['year'].fillna(date_life[0]).astype('int')
df['epi_week'] = df['epi_week'].fillna(date_life[1]).astype('int')

In [37]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,year,epi_week
0,34978840,20200306.0,2.0,1.0,25.0,1.0,15.0,0,738,NaN,2022,10
1,34978840,20220112.0,2.0,1.0,42.0,3.0,15.0,0,738,NaN,2022,10
2,12149681,20200307.0,2.0,1.0,25.0,1.0,15.0,0,738,NaN,2022,10
3,12149681,20220111.0,2.0,1.0,73.0,6.0,15.0,0,738,NaN,2022,10
4,12211040,20200307.0,2.0,1.0,7.0,0.0,15.0,0,738,NaN,2022,10


### Convertir nuestro dataframe (df) en dask dataframe (ddf) para poder trabajar con el ddf de vacunados

**NOTA**: El dataset de vacunados es demasiado grande como para usarlo como pandas dataframe

In [38]:
df = dd.from_pandas(df, npartitions=3)

### Dataset de vacunados

In [39]:
# Leer el dataset (dask dataframe ddf)
url_vac = '../RawData/TB_VACUNACION_COVID19.csv'
col_vac = ['id_persona','fecha_vacunacion','dosis', 'id_vacuna']
ddf_vac = dd.read_csv(url_vac, sep = ",", usecols = col_vac, dtype = {'fecha_vacunacion':'int32', 
                                                                      'dosis': 'int8',
                                                                      'id_vacuna': 'int8',
                                                                      'id_persona': 'int32'})

In [40]:
ddf_vac.head()

,id_persona,fecha_vacunacion,id_vacuna,dosis
0,13474929,20211127,5,2
1,15456861,20220222,5,3
2,15428123,20211013,1,1
3,17787963,20211014,6,2
4,17873180,20220227,5,1


In [41]:
"""
# Convertir la columna 'fecha_vacunacion' a formato fecha y obtener el año y semana epidemiológica
ddf_vac = ddf_vac.assign(fecha_vacunacion = dd.to_datetime(ddf_vac['fecha_vacunacion'], 
                                                           format = '%Y%m%d', 
                                                           errors="coerce"))
"""
# Renombrar y ordenar columnas
ddf_vac = ddf_vac.rename(columns = {'fecha_vacunacion': 'fecha_vacuna'})
ddf_vac = ddf_vac[['id_persona', 'dosis', 'fecha_vacuna', 'id_vacuna']]

In [42]:
ddf_vac.head()

,id_persona,dosis,fecha_vacuna,id_vacuna
0,13474929,2,20211127,5
1,15456861,3,20220222,5
2,15428123,1,20211013,1
3,17787963,2,20211014,6
4,17873180,1,20220227,5


In [43]:
# TOTAL de vacunas aplicadas
len(ddf_vac)

63386865

### Separar información de dosis recibidas para juntarlos con nuestro dataframe principal (df)

In [44]:
# Personas que recibieron 1era dosis
df_dosis_1 = ddf_vac[ddf_vac.dosis == 1]
df_dosis_1 = df_dosis_1.rename(columns = {'dosis' : 'dosis_1', 
                                          'fecha_vacuna': 'fecha_vacuna_1', 
                                          'id_vacuna': 'id_vacuna_1'})
len(df_dosis_1)

27948872

In [45]:
# Unir a nuestro df principal
df = df.merge(df_dosis_1, on = 'id_persona', how = 'left')
del df_dosis_1

In [46]:
# Personas que recibieron 2nda dosis
df_dosis_2 = ddf_vac[ddf_vac.dosis == 2]
df_dosis_2['dosis'] = df_dosis_2['dosis'].replace(2, 1)
df_dosis_2 = df_dosis_2.rename(columns = {'dosis' : 'dosis_2', 
                                          'fecha_vacuna': 'fecha_vacuna_2', 
                                          'id_vacuna': 'id_vacuna_2'})
len(df_dosis_2)

24789831

In [47]:
# Unir a nuestro df principal
df = df.merge(df_dosis_2, on = 'id_persona', how = 'left')
del df_dosis_2

In [48]:
# Personas que recibieron 3era dosis
df_dosis_3 = ddf_vac[ddf_vac.dosis == 3]
df_dosis_3['dosis'] = df_dosis_3['dosis'].replace(3, 1)
df_dosis_3 = df_dosis_3.rename(columns = {'dosis' : 'dosis_3', 
                                          'fecha_vacuna': 'fecha_vacuna_3', 
                                          'id_vacuna': 'id_vacuna_3'})
len(df_dosis_3)

10647688

In [49]:
# Unir a nuestro df principal
df = df.merge(df_dosis_3, on = 'id_persona', how = 'left')   
del df_dosis_3, ddf_vac

In [37]:
# TOTAL de casos positivos más vacunación
len(df)

3465060

### Convertir nuestro ddf en un pandas dataframe

In [ ]:
# IMPORTANTE: Verificar cantidad de RAM disponible antes de ejecutar esta celda
df = df.compute()

In [ ]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,...,epi_week,dosis_1,fecha_vacuna_1,id_vacuna_1,dosis_2,fecha_vacuna_2,id_vacuna_2,dosis_3,fecha_vacuna_3,id_vacuna_3
0,10629488,20200307,2,1,51,4,15.0,0,727,NaN,...,10,NaN,NaN,NaN,NaN,NaN,NaN,1.0,20211129.0,5.0
1,5786491,20200315,2,2,39,2,15.0,0,727,NaN,...,10,1.0,20210823.0,6.0,1.0,20210918.0,6.0,1.0,20220112.0,5.0
2,9040259,20200317,2,1,28,1,15.0,0,727,NaN,...,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6890212,20200317,2,1,35,2,15.0,0,727,NaN,...,10,1.0,20210306.0,6.0,1.0,20210327.0,6.0,1.0,20211114.0,5.0
4,32018834,20200317,2,2,27,1,15.0,0,727,NaN,...,10,1.0,20210213.0,6.0,1.0,20210306.0,6.0,1.0,20211221.0,5.0


In [ ]:
# Personas con 1 dosis recibida en el dataset de casos positivos
len(df[df.dosis_1 == 1])

3145989

In [ ]:
# Personas con 2 dosis recibidas en el dataset de casos positivos
len(df[df.dosis_2 == 1])

3016308

In [ ]:
# Personas con 3 dosis recibidas en el dataset de casos positivos
len(df[df.dosis_3 == 1])

1751123

In [ ]:
# Verificamos que se encuentren todas las columnas
df.columns

Index(['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat',
       'departamento', 'fallecido', 'time_days', 'fecha_fallecimiento', 'year',
       'epi_week', 'dosis_1', 'fecha_vacuna_1', 'id_vacuna_1', 'dosis_2',
       'fecha_vacuna_2', 'id_vacuna_2', 'dosis_3', 'fecha_vacuna_3',
       'id_vacuna_3'],
      dtype='object')

### Rellenar espacios vacíos con 0

In [ ]:
# Crear una columna llamado 'vacunado' si recibe 2 o 3 dosis
df['vacunado'] = np.where((df['dosis_2'] == 1) | (df['dosis_3'] == 1), 1, 0).astype('int8')

In [ ]:
# Convertir de float a int para ocupar menos espacio en memoria
df['dosis_1'] = df['dosis_1'].fillna(0).astype('int8')
df['dosis_2'] = df['dosis_2'].fillna(0).astype('int8')
df['dosis_3'] = df['dosis_3'].fillna(0).astype('int8')

df['id_vacuna_1'] = df['id_vacuna_1'].fillna(0).astype('int8')
df['id_vacuna_2'] = df['id_vacuna_2'].fillna(0).astype('int8')
df['id_vacuna_3'] = df['id_vacuna_3'].fillna(0).astype('int8')

In [ ]:
# Verificamos que la columna vacunados tenga en cuenta 2 o 3 dosis como persona vacunada
df[['dosis_1', 'dosis_2', 'dosis_3', 'vacunado']].head(10)

,dosis_1,dosis_2,dosis_3,vacunado
0,0,0,1,1
1,1,1,1,1
2,0,0,0,0
3,1,1,1,1
4,1,1,1,1
5,0,0,1,1
6,1,1,0,1
7,1,1,0,1
8,1,1,0,1
9,1,1,1,1


### Ordenar y Guardar el df

In [ ]:
df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento', 
         'fallecido', 'time_days', 'fecha_fallecimiento', 'year', 'epi_week',
         'vacunado', 'dosis_1', 'dosis_2', 'dosis_3', 
         'id_vacuna_1', 'id_vacuna_2', 'id_vacuna_3' ,'fecha_vacuna_1','fecha_vacuna_2', 'fecha_vacuna_3']]

### Eliminar duplicados

In [ ]:
# Ordenar de acuerdo al id_de persona y fecha de resultado
df = df.sort_values(by = ['id_persona', 'fecha_resultado'])

In [ ]:
# Vemos todos los duplicados, ya que debería haber un solo id para cada persona
df[df.id_persona.duplicated() == True]

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,...,vacunado,dosis_1,dosis_2,dosis_3,id_vacuna_1,id_vacuna_2,id_vacuna_3,fecha_vacuna_1,fecha_vacuna_2,fecha_vacuna_3
69732,47,20220129,2,1,62,5,25.0,0,727,NaN,...,1,1,1,1,1,2,5,20210506.0,20210608.0,20211127.0
69794,89,20220129,2,2,74,6,25.0,0,727,NaN,...,1,1,1,0,1,2,0,20210426.0,20210609.0,NaN
37557,227,20210421,3,1,81,7,25.0,0,727,NaN,...,1,1,1,0,6,6,0,20211202.0,20211223.0,NaN
41765,324,20210525,3,2,54,4,25.0,0,727,NaN,...,1,1,1,1,2,1,5,20210824.0,20210927.0,20220106.0
68905,555,20220127,2,2,55,4,25.0,0,727,NaN,...,1,1,1,1,2,2,5,20210712.0,20210810.0,20211228.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57061,37278364,20220116,3,2,1,0,20.0,0,727,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
31454,37281588,20210321,2,1,19,1,15.0,0,727,NaN,...,0,1,0,0,5,0,0,20220208.0,NaN,NaN
53381,37285950,20220112,2,1,26,1,20.0,1,54,20200506.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
53720,37286078,20220112,2,2,22,1,15.0,1,79,20200531.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


Podremos conservar el último duplicado, ya es es la fecha más reciente de un caso positivo. Estos duplicados representan varios contagios de la misma persona, sin embargo se toman como un nuevo contagio.

En este caso para el propósito del estudio nos interesa la mortalidad y la vacunación, por ello no se toma en cuenta el número de veces de contagio o fechas de la misma, si no todos los datos e id del paciente.

In [ ]:
# Eliminamos los duplicados
df = df.drop_duplicates('id_persona', keep='last') \
       .sort_values(by = ['fecha_resultado']) \
       .reset_index(drop = True)

In [ ]:
df.head(10)

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,...,vacunado,dosis_1,dosis_2,dosis_3,id_vacuna_1,id_vacuna_2,id_vacuna_3,fecha_vacuna_1,fecha_vacuna_2,fecha_vacuna_3
0,13708801,20200309,2,2,15,0,10.0,0,727,NaN,...,1,1,1,1,5,5,5,20210601.0,20210622.0,20211203.0
1,2020432,20200310,2,1,46,3,15.0,0,727,NaN,...,1,1,1,1,5,5,5,20210517.0,20210607.0,20211206.0
2,5493803,20200311,2,2,28,1,15.0,0,727,NaN,...,1,0,0,1,0,0,5,NaN,NaN,20220104.0
3,7979960,20200311,2,2,21,1,15.0,0,727,NaN,...,1,1,1,0,6,6,0,20210926.0,20211019.0,NaN
4,27127031,20200312,2,1,24,1,15.0,0,727,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
5,35668643,20200312,2,2,39,2,15.0,0,727,NaN,...,1,0,0,1,0,0,5,NaN,NaN,20211201.0
6,10960790,20200312,2,2,58,4,15.0,0,727,NaN,...,1,1,1,1,5,5,5,20210625.0,20210716.0,20211222.0
7,4102088,20200312,2,2,37,2,15.0,0,727,NaN,...,1,1,1,1,6,6,5,20210918.0,20211009.0,20220127.0
8,36889651,20200312,2,2,75,6,15.0,1,7,20200320.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
9,36611013,20200312,2,1,78,6,15.0,1,6,20200319.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [ ]:
# Verificamos columnas y data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3247360 entries, 0 to 3247359
Data columns (total 22 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id_persona           int32  
 1   fecha_resultado      int32  
 2   metododx             int8   
 3   sexo                 int8   
 4   edad                 int8   
 5   edad_cat             int8   
 6   departamento         float64
 7   fallecido            int8   
 8   time_days            int64  
 9   fecha_fallecimiento  float64
 10  year                 int64  
 11  epi_week             int64  
 12  vacunado             int8   
 13  dosis_1              int8   
 14  dosis_2              int8   
 15  dosis_3              int8   
 16  id_vacuna_1          int8   
 17  id_vacuna_2          int8   
 18  id_vacuna_3          int8   
 19  fecha_vacuna_1       float64
 20  fecha_vacuna_2       float64
 21  fecha_vacuna_3       float64
dtypes: float64(5), int32(2), int64(3), int8(12)
memory usage: 260.1 MB

In [ ]:
# Guardamos en un csv
df.to_csv('../Data/DP3_covid19-peru_casos-positivos_+_info-fallecimiento_+_info-vacunas.csv',  index=False)

In [ ]:
# Eliminamos el dataframe en memoria
del df

In [39]:
df = pd.read_csv('../Data/DP3_covid19-peru_casos-positivos_+_info-fallecimiento_+_info-vacunas.csv')